# Overview

Import file into local database

# Setup

Configure the environment variables. Copy `example.env` to `.env` and update the values
```bash
cp example.env .env

# load .env vars (optional)
[ -f .env ] && while IFS= read -r line; do [[ $line =~ ^[^#]*= ]] && eval "export $line"; done < .env
```

In [1]:
import pandas as pd
import pathlib
import os

from dotenv import load_dotenv
load_dotenv()

project_root = os.path.dirname(os.getcwd())
reports_dir = os.getenv("DATA_DIR")
file_name = os.getenv("TEST_FILE_NAME")
csv_file = os.path.join(project_root, reports_dir, file_name)
csv_file

'/workspaces/AzureBillingReports/temp/Azure_Actual_20230501-20230531_csv_Azure_ActualCost_v20230605T004903Z_574afa1e-0724-4291-a5f9-e82743b07f23/part_01.csv'

## Create local database

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import pathlib

csv_database = create_engine('sqlite:///small_sample.db')

## Import to Database

In [5]:
from billing import BILLING_DTYPES, DEFAULT_CHUNK_SIZE, BILLING_COLUMN_NAMES

print(BILLING_COLUMN_NAMES)
batch_no=1
csv_iterator = pd.read_csv(csv_file, chunksize=DEFAULT_CHUNK_SIZE, header=None, names=BILLING_COLUMN_NAMES, iterator=True, parse_dates=['Date', 'BillingPeriodStartDate', 'BillingPeriodEndDate'])
for chunk in csv_iterator:
    chunk.to_sql('ActualCost',csv_database, if_exists='append')
    batch_no+=1
    print('index: {}'.format(batch_no))

['InvoiceSectionName', 'AccountName', 'AccountOwnerId', 'SubscriptionId', 'SubscriptionName', 'ResourceGroup', 'ResourceLocation', 'Date', 'ProductName', 'MeterCategory', 'MeterSubCategory', 'MeterId', 'MeterName', 'MeterRegion', 'UnitOfMeasure', 'Quantity', 'EffectivePrice', 'CostInBillingCurrency', 'CostCenter', 'ConsumedService', 'ResourceId', 'Tags', 'OfferId', 'AdditionalInfo', 'ServiceInfo1', 'ServiceInfo2', 'ResourceName', 'ReservationId', 'ReservationName', 'UnitPrice', 'ProductOrderId', 'ProductOrderName', 'Term', 'PublisherType', 'PublisherName', 'ChargeType', 'Frequency', 'PricingModel', 'AvailabilityZone', 'BillingAccountId', 'BillingAccountName', 'BillingCurrencyCode', 'BillingPeriodStartDate', 'BillingPeriodEndDate', 'BillingProfileId', 'BillingProfileName', 'InvoiceSectionId', 'IsAzureCreditEligible', 'PartNumber', 'PayGPrice', 'PlanName', 'ServiceFamily', 'CostAllocationRuleName', 'benefitId', 'benefitName']


/tmp/ipykernel_46742/1115165564.py:6: DtypeWarning: Columns (27,28,30,31,34,50,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in csv_iterator:


index: 2


In [7]:
df_new = pd.read_sql_query('SELECT Date, Sum(CostInBillingCurrency) FROM ActualCost Group By Date',csv_database)
df_new.head()

,Date,Sum(CostInBillingCurrency)
0,2023-05-01 00:00:00.000000,264993.057861
1,2023-05-02 00:00:00.000000,138322.067972
